## Retriever and Chain with Langchain

In [5]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("bhutan_trip.pdf")
docs = loader.load()
docs

[Document(page_content='ALL HIMALAYA TOURS BHUTAN\n                Babesa: Thimphu\n Email:allhimalayantours@gmail.com\n            Contact No. +97577759316\nCultural Tour Package itinerary-6 Days 5 Nights\nAfter your arrival to Phuntsholing, your guide and driver will welcome you and then take a long drive \ntowards Thimphu the capital city of Bhutan. The distance is approximately 174 kms and goes across spectac -\nular landscapes with waterfalls,small villages and towns and with constantly changing vegetation. After we \narrive in Thimphu we will check in the hotel and enjoy your first night in Bhutan.\nPhuntsholingDay1', metadata={'source': 'bhutan_trip.pdf', 'page': 0}),
 Document(page_content='After breakfast, you will visit the National Memorial Chorten which is the most visible religious landmark in \nBhutan. It was built in 1974 in the memory of the third  king of Bhutan Jigme Dorji Wangchuck. Thereafter, \nwe will drive to Kuensel Phodrang National Park where you will see a 16

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
text_splitter.split_documents(docs)[:5]

[Document(page_content='ALL HIMALAYA TOURS BHUTAN\n                Babesa: Thimphu\n Email:allhimalayantours@gmail.com\n            Contact No. +97577759316\nCultural Tour Package itinerary-6 Days 5 Nights\nAfter your arrival to Phuntsholing, your guide and driver will welcome you and then take a long drive \ntowards Thimphu the capital city of Bhutan. The distance is approximately 174 kms and goes across spectac -\nular landscapes with waterfalls,small villages and towns and with constantly changing vegetation. After we \narrive in Thimphu we will check in the hotel and enjoy your first night in Bhutan.\nPhuntsholingDay1', metadata={'source': 'bhutan_trip.pdf', 'page': 0}),
 Document(page_content='After breakfast, you will visit the National Memorial Chorten which is the most visible religious landmark in \nBhutan. It was built in 1974 in the memory of the third  king of Bhutan Jigme Dorji Wangchuck. Thereafter, \nwe will drive to Kuensel Phodrang National Park where you will see a 16

In [7]:
documents = text_splitter.split_documents(docs)
documents

[Document(page_content='ALL HIMALAYA TOURS BHUTAN\n                Babesa: Thimphu\n Email:allhimalayantours@gmail.com\n            Contact No. +97577759316\nCultural Tour Package itinerary-6 Days 5 Nights\nAfter your arrival to Phuntsholing, your guide and driver will welcome you and then take a long drive \ntowards Thimphu the capital city of Bhutan. The distance is approximately 174 kms and goes across spectac -\nular landscapes with waterfalls,small villages and towns and with constantly changing vegetation. After we \narrive in Thimphu we will check in the hotel and enjoy your first night in Bhutan.\nPhuntsholingDay1', metadata={'source': 'bhutan_trip.pdf', 'page': 0}),
 Document(page_content='After breakfast, you will visit the National Memorial Chorten which is the most visible religious landmark in \nBhutan. It was built in 1974 in the memory of the third  king of Bhutan Jigme Dorji Wangchuck. Thereafter, \nwe will drive to Kuensel Phodrang National Park where you will see a 16

In [9]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

db = FAISS.from_documents(documents, OllamaEmbeddings())

In [11]:
query = "Where is Taktsang Monaestry?"
result = db.similarity_search(query)

result[0].page_content

'Are you ready? We will be making the hike up to one of the key highlights in Bhutan\n today!\nTiger’s Nest aka Taktsang Monastery. Taktsang Monastery built on a 900 metres high cliff is the most popular \nsite of Bhutan and one hour hike to the cafeteria is also a vantage view whereby you can enjoy the stunning \nview of the monastery. Prayer flags adorn the cliffs and this is also where Guru Padmasambhava landed on \nthe back of a tigress in the 8th century.\nIn the evening, we will have an opportunity to relax your tired body muscles in a traditional hotstone bath \nand enjoy traditional cultural programs upon your interests.\nIn the evening, you can walk around Paro town and buy souvenirs for your friends and families. Night halt at \na Hotel in Paro.\nTaktshang Monastry or the Tiger’s Nest Monastry\nTraditional hotstone bath Traditional cultural programDay4'

In [12]:
from langchain_community.llms import Ollama

## Load Ollama Llama 2 model
llm = Ollama(model="llama2")
llm

Ollama()

## Prompt

In [13]:
## Design the ChatPrompt Template
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""
    Answer the following question based only on the provided context.
    Think step by step before providing a detailed answer.
    I will tip you $1000 if the user finds the answer helpful.
    <context>
    {context}
    </context>
    Question: {input}""")

## Chain

In [15]:
## Create Stuff Document Chain
## Info: https://python.langchain.com/v0.1/docs/modules/chains/

'''Note: prompt (BasePromptTemplate) Prompt template. Must contain 
input variable “context”, which will be used for passing in the 
formatted documents.'''
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain=create_stuff_documents_chain(llm, prompt)

## Retriever

In [17]:
## Retriever
## Info: https://python.langchain.com/v0.1/docs/modules/data_connection/retrievers/
retriever = db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001ECE933AB80>)

## Create Retriever Chain

In [18]:
"""
Retriever Chain: This chain takes in a user inquiry, which is then 
passed to the retriever to fetch relevant documents. Those documents
(and original inputs) are then passed to an LLM to generate a response.
"""
from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [20]:
response = retrieval_chain.invoke({"input": "Which city is Taktsang Monaestry located?"})

In [21]:
response

{'input': 'Which city is Taktsang Monaestry located?',
 'context': [Document(page_content='Are you ready? We will be making the hike up to one of the key highlights in Bhutan\n today!\nTiger’s Nest aka Taktsang Monastery. Taktsang Monastery built on a 900 metres high cliff is the most popular \nsite of Bhutan and one hour hike to the cafeteria is also a vantage view whereby you can enjoy the stunning \nview of the monastery. Prayer flags adorn the cliffs and this is also where Guru Padmasambhava landed on \nthe back of a tigress in the 8th century.\nIn the evening, we will have an opportunity to relax your tired body muscles in a traditional hotstone bath \nand enjoy traditional cultural programs upon your interests.\nIn the evening, you can walk around Paro town and buy souvenirs for your friends and families. Night halt at \na Hotel in Paro.\nTaktshang Monastry or the Tiger’s Nest Monastry\nTraditional hotstone bath Traditional cultural programDay4', metadata={'source': 'bhutan_trip.

In [22]:
response["answer"]

'Based on the provided context, Taktsang Monastery is located in Paro.'